Импортим библиотеки

In [21]:
import csv
import sys
import pandas as pd
!pip install telethon
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install webdriver-manager
from telethon.sync import TelegramClient
from telethon.tl.types import Message
import asyncio
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,000 kB]
Fetched 4,919 kB in 2s (2,473 kB/s)
Reading package lists... Done
W: Skipping acquire of

Функция для выгрузки данных из телеграмм каналов

In [40]:
async def export_channel_posts(channel_username, api_id, api_hash, csv_filename='posts.csv'):
    async with TelegramClient('session_name', api_id, api_hash) as client:
        posts = []

        async for message in client.iter_messages(channel_username):
            if isinstance(message, Message) and message.message:
                posts.append([message.id, message.date.strftime("%Y-%m-%d %H:%M:%S"), message.message])

        with open(csv_filename, 'w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['id', 'date', 'text'])
            writer.writerows(posts)

Выгружаем данные

In [3]:
channels=['https://t.me/infomoscow24', 'https://t.me/mos_sobyanin', 'https://t.me/interfaxonline']

In [41]:
API_ID=''
API_HASH=''

In [20]:
for i in range(len(channels)):
  await export_channel_posts(channels[i],API_ID,API_HASH,str(i)+'posts.csv')

Please enter your phone (or bot token): +79859644955
Please enter the code you received: 81901
Please enter your password: ··········
Signed in successfully as Kirill Dolgopol; remember to not break the ToS or you will risk an account ban!


Оставляем только новости, касающиеся парковок

In [31]:
def filter_csv_files(filenames, new_filename, filter_str):
    import pandas as pd
    filtered_dfs = []
    for file in filenames:
        df = pd.read_csv(file)
        mask = df.iloc[:, 2].str.contains(filter_str, case=False, na=False)
        filtered_dfs.append(df[mask])
    result_df = pd.concat(filtered_dfs, ignore_index=True)
    result_df.to_csv(new_filename, index=False, encoding="utf-8-sig")

In [29]:
filenames = ["0posts.csv", "1posts.csv", "2posts.csv"]

In [32]:
filter_csv_files(filenames,'parkovki.csv',r"парковоч|парковк")

Теперь смотрим еслть ли там где-то что-то связанное с сервисом, приложением и тд

In [33]:
filenames = ["parkovki.csv"]

In [34]:
filter_csv_files(filenames,'servisi.csv',r"сервис|прилож")

Теперь смотрим что-то про свободные места

In [36]:
filenames = ["servisi.csv"]

In [37]:
filter_csv_files(filenames,'mesta.csv',r"свобод|пуст|занят")

Ручным просмотром новостей находим потенциальных конкурентов: сервисы "Парковки России" и "Электронный дом"

Смотрим все объявления с этими сервисами в разделе парковки

In [38]:
filter_csv_files(['parkovki.csv'],'parkovkirossii.csv',r"Парковки России|парковки России")

In [39]:
filter_csv_files(['parkovki.csv'],'elecdom.csv',r"Электронный дом|электронный дом|Электронный Дом")